In [1]:
import itertools
from parlay_system import *
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [2]:
# Dec 21 games

# NOTE: Odds boosted to +105 for a patriots, chiefs, sf49ers outcome
notredame = MoneyLine(event="notredame", bet_amount=100, odds=260)
indiana = MoneyLine(event="indiana", bet_amount=100, odds=-335)

missouri = MoneyLine(event="missouri", bet_amount=100, odds=195)
illinois = MoneyLine(event="illinois", bet_amount=100, odds=-245)

texasst = MoneyLine(event="texasst", bet_amount=100, odds=140)
georgiast = MoneyLine(event="georgiast", bet_amount=100, odds=-175)

binaries = [
    [notredame, indiana],
    [missouri, illinois],
    [texasst, georgiast]
]

results = [
    [0, 1], #[towson, liberty],
    [0, 1], #[ucirvine, illchi],
    [0, 1], #[wizards, sf49ers],
]

override_arr = [0, 2, 99]


def create_dicts(in_binaries, in_results):
    ml_lookup = {}
    ml_win_lookup = {}
    id = 0
    for bi in range(len(in_binaries)):
        binary = in_binaries[bi]
        for ml_i in range(len(binary)):
            ml = binary[ml_i]
            ml.set_index(id) 
            is_winner = in_results[bi][ml_i]
            ml_lookup[id] = ml.event
            ml_win_lookup[id] = is_winner
            id += 1
    
    return ml_lookup, ml_win_lookup

ml_lookup, ml_outcome_lookup = create_dicts(binaries, results)

In [3]:
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

TOURNEY_SIZE = len(binaries)
SELECT_NUM = 3
s = set()
n = SELECT_NUM

for i in range(TOURNEY_SIZE):
    s.add(i)

all_subsets = findsubsets(s, n)

In [4]:
len(all_subsets)

1

In [5]:
csv = []
df = pd.DataFrame()

for i in range(len(all_subsets)):
    current_subset = all_subsets[i]
    parlay_binaries = []
    for i in range(len(current_subset)):
        index = current_subset[i]
        parlay_binaries.append(binaries[index])

    ps = ParlaySystem(binaries=parlay_binaries,
                    target_profit=0.15,
                    bounds=(0.01, 30),
                    binary_index_arr=list(current_subset),
                    binary_results_arr=results,
                    index_to_ml=ml_lookup,
                    index_to_outcome=ml_outcome_lookup,
                    override_arr=override_arr
                    )
    
    csv_res, raw_df = ps.slsqp_solver()
    df = pd.concat([df, raw_df])
    csv.append(csv_res)

     fun: 4.917586373028485
     jac: array([-2.31099999, -1.21107143, -0.64581633, -0.1207726 , -0.2741791 ,
        0.12256134,  0.32644689,  0.51582831])
 message: 'Positive directional derivative for linesearch'
    nfev: 75
     nit: 10
    njev: 6
  status: 8
 success: False
       x: array([0.01541842, 0.02354814, 0.03230048, 0.04773875, 0.04274625,
       0.04085297, 0.03037977, 0.01      ])
slsqp_solver: 
                          event    index[]     result  event_status         odds    bet       mult  payout  profit
7    indiana_illinois_georgiast  [1, 3, 5]  [1, 1, 1]          True   187.337366  0.010   2.873374  0.0287  -0.214
0    notredame_missouri_texasst  [0, 2, 4]  [0, 0, 0]         False  2448.800000  0.015  25.488000  0.3930   0.150
1  notredame_missouri_georgiast  [0, 2, 5]  [0, 0, 1]         False  1568.857143  0.024  16.688571  0.3930   0.150
2    notredame_illinois_texasst  [0, 3, 4]  [0, 1, 0]         False  1116.653061  0.032  12.166531  0.3930   0.150
4      

In [6]:
df.describe()

,odds,bet,mult,payout,profit
count,8.000000,8.000000,8.000000,8.000000,8.000000
mean,959.800301,0.030375,10.598003,0.295025,0.052000
std,745.805390,0.013511,7.458054,0.144007,0.143945
min,187.337366,0.010000,2.873374,0.028700,-0.214000
25%,461.173850,0.021750,5.611739,0.217750,-0.025250
50%,757.980680,0.031000,8.579807,0.386650,0.143500
75%,1229.704082,0.041500,13.297041,0.393000,0.150000
max,2448.800000,0.048000,25.488000,0.393000,0.150000


In [7]:
negative_df = df.loc[df.profit < 0]
negative_df = negative_df.sort_values(by=['event_status', 'profit'], ascending=[False, True])
negative_df.describe()

,odds,bet,mult,payout,profit
count,2.000000,2.000000,2.000000,2.000000,2.000000
mean,263.089944,0.020000,3.630899,0.081000,-0.162000
std,107.130323,0.014142,1.071303,0.073963,0.073539
min,187.337366,0.010000,2.873374,0.028700,-0.214000
25%,225.213655,0.015000,3.252137,0.054850,-0.188000
50%,263.089944,0.020000,3.630899,0.081000,-0.162000
75%,300.966233,0.025000,4.009662,0.107150,-0.136000
max,338.842522,0.030000,4.388425,0.133300,-0.110000


In [8]:
positive_df = df.loc[df.profit >= 0]
positive_df = positive_df.sort_values(by=['event_status', 'profit'], ascending=[False, False])
positive_df.describe()

,odds,bet,mult,payout,profit
count,6.000000,6.000000,6.000000,6.000000,6.000000
mean,1192.037087,0.033833,12.920371,0.366367,0.123333
std,719.400953,0.012576,7.194010,0.059235,0.059180
min,501.950959,0.015000,6.019510,0.245900,0.003000
25%,727.299378,0.026000,8.272994,0.383475,0.140250
50%,967.998172,0.036500,10.679982,0.393000,0.150000
75%,1455.806122,0.042500,15.558061,0.393000,0.150000
max,2448.800000,0.048000,25.488000,0.393000,0.150000


In [9]:
negative_df.head(200)

,event,index[],result,event_status,odds,bet,mult,payout,profit
7,indiana_illinois_georgiast,"[1, 3, 5]","[1, 1, 1]",True,187.337366,0.01,2.873374,0.0287,-0.214
6,indiana_illinois_texasst,"[1, 3, 4]","[1, 1, 0]",False,338.842522,0.03,4.388425,0.1333,-0.110


In [10]:
positive_df.head(200)

,event,index[],result,event_status,odds,bet,mult,payout,profit
0,notredame_missouri_texasst,"[0, 2, 4]","[0, 0, 0]",False,2448.800000,0.015,25.488000,0.3930,0.150
1,notredame_missouri_georgiast,"[0, 2, 5]","[0, 0, 1]",False,1568.857143,0.024,16.688571,0.3930,0.150
2,notredame_illinois_texasst,"[0, 3, 4]","[0, 1, 0]",False,1116.653061,0.032,12.166531,0.3930,0.150
4,indiana_missouri_texasst,"[1, 2, 4]","[1, 0, 0]",False,819.343284,0.043,9.193433,0.3930,0.150
3,notredame_illinois_georgiast,"[0, 3, 5]","[0, 1, 1]",False,696.618076,0.048,7.966181,0.3803,0.137
5,indiana_missouri_georgiast,"[1, 2, 5]","[1, 0, 1]",False,501.950959,0.041,6.019510,0.2459,0.003


In [11]:
positive_df.groupby('event_status').count()

,event,index[],result,odds,bet,mult,payout,profit
event_status,,,,,,,,
False,6,6,6,6,6,6,6,6


In [12]:
negative_df.groupby('event_status').count()

,event,index[],result,odds,bet,mult,payout,profit
event_status,,,,,,,,
False,1,1,1,1,1,1,1,1
True,1,1,1,1,1,1,1,1


In [13]:
neg = negative_df.reset_index(drop=True).style.applymap(color_negative_red)
neg

,event,index[],result,event_status,odds,bet,mult,payout,profit
0,indiana_illinois_georgiast,"['1', '3', '5']","[1, 1, 1]",True,187.337,0.01,2.87337,0.0287,-0.214
1,indiana_illinois_texasst,"['1', '3', '4']","[1, 1, 0]",False,338.843,0.03,4.38843,0.1333,-0.11


In [14]:
pos = positive_df.reset_index(drop=True).style.applymap(color_positive_green)
pos

,event,index[],result,event_status,odds,bet,mult,payout,profit
0,notredame_missouri_texasst,"['0', '2', '4']","[0, 0, 0]",False,2448.8,0.015,25.488,0.393,0.15
1,notredame_missouri_georgiast,"['0', '2', '5']","[0, 0, 1]",False,1568.86,0.024,16.6886,0.393,0.15
2,notredame_illinois_texasst,"['0', '3', '4']","[0, 1, 0]",False,1116.65,0.032,12.1665,0.393,0.15
3,indiana_missouri_texasst,"['1', '2', '4']","[1, 0, 0]",False,819.343,0.043,9.19343,0.393,0.15
4,notredame_illinois_georgiast,"['0', '3', '5']","[0, 1, 1]",False,696.618,0.048,7.96618,0.3803,0.137
5,indiana_missouri_georgiast,"['1', '2', '5']","[1, 0, 1]",False,501.951,0.041,6.01951,0.2459,0.003


In [15]:
sf49ers = MoneyLine(event="parlay", bet_amount=0.01, odds=110)
sf49ers.print_stats()

    event  bet_amount  odds  multiplier  payout
0  parlay        0.01   110         2.1   0.021


In [16]:
pos = MoneyLine(event="pos", bet_amount=1.0, odds=245)
pos.print_stats()

  event  bet_amount  odds  multiplier  payout
0   pos         1.0   245        3.45    3.45


In [17]:
neg = MoneyLine(event="neg", bet_amount=1.0, odds=-305)
neg.print_stats()

  event  bet_amount  odds  multiplier    payout
0   neg         1.0  -305    1.327869  1.327869


In [18]:
neg = MoneyLine(event="neg", bet_amount=1.0, odds=-285)
neg.print_stats()

  event  bet_amount  odds  multiplier    payout
0   neg         1.0  -285    1.350877  1.350877
